In [4]:
import itertools
import math
from tkinter import *
from tkinter import filedialog

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [5]:
def adp_filt(currdf: object, pose):
    lIndex = []
    xIndex = []
    yIndex = []
    currdf = np.array(currdf[1:])
    for header in pose:
        if currdf[0][header + 1] == "likelihood":
            lIndex.append(header)
        elif currdf[0][header + 1] == "x":
            xIndex.append(header)
        elif currdf[0][header + 1] == "y":
            yIndex.append(header)
    curr_df1 = currdf[:, 1:]
    datax = curr_df1[1:, np.array(xIndex)]
    datay = curr_df1[1:, np.array(yIndex)]
    data_lh = curr_df1[1:, np.array(lIndex)]
    currdf_filt = np.zeros((datax.shape[0], (datax.shape[1]) * 2))
    perc_rect = []
    for i in range(data_lh.shape[1]):
        perc_rect.append(0)
    for x in tqdm(range(data_lh.shape[1])):
        a, b = np.histogram(data_lh[1:, x].astype(np.float64))
        rise_a = np.where(np.diff(a) >= 0)
        if rise_a[0][0] > 1:
            llh = b[rise_a[0][0]]
        else:
            llh = b[rise_a[0][1]]
        # llh=0
        data_lh_float = data_lh[:, x].astype(np.float64)
        perc_rect[x] = np.sum(data_lh_float < llh) / data_lh.shape[0]
        currdf_filt[0, (2 * x):(2 * x + 2)] = np.hstack([datax[0, x], datay[0, x]])
        for i in range(1, data_lh.shape[0]):
            if data_lh_float[i] < llh:
                currdf_filt[i, (2 * x):(2 * x + 2)] = currdf_filt[i - 1, (2 * x):(2 * x + 2)]
            else:
                currdf_filt[i, (2 * x):(2 * x + 2)] = np.hstack([datax[i, x], datay[i, x]])
    currdf_filt = np.array(currdf_filt)
    currdf_filt = currdf_filt.astype(np.float64)
    return currdf_filt, perc_rect

def boxcar_center(a, n):
    a1 = pd.Series(a)
    moving_avg = np.array(a1.rolling(window=n, min_periods=1, center=True).mean())

    return moving_avg


def feature_extraction(processed_input_data, framerate = 30):
    window = np.int64(np.round(0.05 / (1 / framerate)) * 2 - 1)
    f = []
    for n in range(len(processed_input_data)):
        data_n_len = len(processed_input_data[n])
        dxy_list = []
        disp_list = []
        for r in range(data_n_len):
            if r < data_n_len - 1:
                disp = []
                for c in range(0, processed_input_data[n].shape[1], 2):
                    disp.append(
                        np.linalg.norm(processed_input_data[n][r + 1, c:c + 2] -
                                       processed_input_data[n][r, c:c + 2]))
                disp_list.append(disp)
            dxy = []
            for i, j in itertools.combinations(range(0, processed_input_data[n].shape[1], 2), 2):
                dxy.append(processed_input_data[n][r, i:i + 2] -
                           processed_input_data[n][r, j:j + 2])
            dxy_list.append(dxy)
        disp_r = np.array(disp_list)
        dxy_r = np.array(dxy_list)
        disp_boxcar = []
        dxy_eu = np.zeros([data_n_len, dxy_r.shape[1]])
        ang = np.zeros([data_n_len - 1, dxy_r.shape[1]])
        dxy_boxcar = []
        ang_boxcar = []
        for l in range(disp_r.shape[1]):
            disp_boxcar.append(boxcar_center(disp_r[:, l], window))
            # disp_boxcar.append(disp_r[:, l])
        for k in range(dxy_r.shape[1]):
            for kk in range(data_n_len):
                dxy_eu[kk, k] = np.linalg.norm(dxy_r[kk, k, :])
                if kk < data_n_len - 1:
                    b_3d = np.hstack([dxy_r[kk + 1, k, :], 0])
                    a_3d = np.hstack([dxy_r[kk, k, :], 0])
                    c = np.cross(b_3d, a_3d)
                    ang[kk, k] = np.dot(np.dot(np.sign(c[2]), 180) / np.pi,
                                        math.atan2(np.linalg.norm(c),
                                                   np.dot(dxy_r[kk, k, :], dxy_r[kk + 1, k, :])))
            dxy_boxcar.append(boxcar_center(dxy_eu[:, k], window))
            ang_boxcar.append(boxcar_center(ang[:, k], window))
            # dxy_boxcar.append(dxy_eu[:, k])
            # ang_boxcar.append(ang[:, k])
        disp_feat = np.array(disp_boxcar)
        dxy_feat = np.array(dxy_boxcar)
        ang_feat = np.array(ang_boxcar)
        f.append(np.vstack((dxy_feat[:, 1:], ang_feat, disp_feat)))

    for m in range(0, len(f)):
        f_integrated = np.zeros(len(processed_input_data[m]))
        for k in range(round(framerate / 10), len(f[m][0]), round(framerate / 10)):
            if k > round(framerate / 10):
                f_integrated = np.concatenate(
                    (f_integrated.reshape(f_integrated.shape[0], f_integrated.shape[1]),
                     np.hstack((np.mean((f[m][0:dxy_feat.shape[0],
                                         range(k - round(framerate / 10), k)]), axis=1),
                                np.sum((f[m][dxy_feat.shape[0]:f[m].shape[0],
                                        range(k - round(framerate / 10), k)]), axis=1)
                                )).reshape(len(f[0]), 1)), axis=1
                )
            else:
                f_integrated = np.hstack(
                    (np.mean((f[m][0:dxy_feat.shape[0], range(k - round(framerate / 10), k)]), axis=1),
                     np.sum((f[m][dxy_feat.shape[0]:f[m].shape[0],
                             range(k - round(framerate / 10), k)]), axis=1))).reshape(len(f[0]), 1)
        if m > 0:
            features = np.concatenate((features, f_integrated), axis=1)
        else:
            features = f_integrated
    return f_integrated

In [9]:
infile = 'Z:\KayCei\VideoOutputCamera 0-27062023-092810DLC_resnet50_bottomup_clearSep21shuffle1_1030000.csv'
df = pd.read_csv(infile)

C:\Users\Yttri-Lab\AppData\Local\Temp\ipykernel_18544\3720120552.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(infile)


In [10]:
df

,scorer,DLC_resnet50_bottomup_clearSep21shuffle1_1030000,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.1,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.2,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.3,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.4,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.5,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.6,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.7,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.8,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.9,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.10,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.11,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.12,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.13,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.14,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.15,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.16,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.17
0,bodyparts,snout,snout,snout,right-forepaw,right-forepaw,right-forepaw,left-forepaw,left-forepaw,left-forepaw,right-hindpaw,right-hindpaw,right-hindpaw,left-hindpaw,left-hindpaw,left-hindpaw,tail-base,tail-base,tail-base
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,683.7763061523438,500.68756103515625,0.9999992847442627,682.2798461914062,543.3304443359375,0.9999940395355225,700.0860595703125,561.3426513671875,0.9998348951339722,663.5510864257812,557.3751220703125,0.9999860525131226,664.8436279296875,601.7637329101562,0.9999971389770508,631.6176147460938,589.6014404296875,0.9999939203262329
3,1,684.24560546875,500.392578125,0.999998927116394,683.3619384765625,542.7285766601562,0.9999885559082031,702.4534912109375,558.818603515625,0.9999836683273315,664.4501953125,557.2879028320312,0.9999616146087646,665.45849609375,602.2630004882812,0.9999973773956299,631.0234985351562,589.3690185546875,0.9999988079071045
4,2,684.6790161132812,500.3468933105469,0.9999992847442627,683.275634765625,543.0970458984375,0.9999860525131226,702.44580078125,558.0687866210938,0.9999896287918091,664.5399169921875,557.9651489257812,0.9999501705169678,665.2615356445312,602.9714965820312,0.9999967813491821,631.1380004882812,589.1498413085938,0.999998927116394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118837,118835,343.998871,595.079041,0.14976,329.193695,586.031677,0.999763,330.713409,595.607788,0.999944,299.266541,568.322632,0.999999,306.718719,603.800842,0.999998,260.935425,569.634155,0.999999
118838,118836,344.303162,594.466248,0.165518,328.777496,585.25293,0.9997,331.973969,595.049133,0.999925,298.638489,568.470459,0.999999,307.432526,604.421143,0.999998,260.625732,569.857422,0.999999
118839,118837,344.075287,595.058777,0.406713,328.589264,585.034119,0.999699,332.068542,594.917664,0.999915,298.72345,568.304077,0.999999,307.172058,604.40033,0.999998,260.606232,570.308167,0.999998
118840,118838,344.174988,594.75647,0.331331,328.698669,585.20105,0.999706,332.049713,594.941345,0.99994,298.805542,568.23407,0.999999,307.120514,604.299316,0.999999,260.647247,569.757751,0.999999


In [11]:
with open('../../../model/clf.pkl', 'rb') as fr:
    clf = pickle.load(fr)

C:\Users\Yttri-Lab\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Yttri-Lab\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [12]:
# filter likelihood (smoothing)
csv_array_filtered, perc_filtered = adp_filt(df, np.arange(18))
f_integrated = feature_extraction([csv_array_filtered.copy()])

100%|██████████| 6/6 [00:04<00:00,  1.31it/s]


In [13]:
X = f_integrated.T.copy()
predictions = clf.predict(X)

In [22]:
# 100ms, so 3000 * 0.1s is actually 300s
class_of_interest = 3 # locomotion
behavior_names = ['left', 'right', 'locomotion', 'face groom', 'other grooms', 'all other']
start = 0 
stop = 3000

first_5min_pred = predictions[start:stop] # first 5 minutes (300*10 at 10fps)
idx_end = np.hstack([np.where(np.diff(first_5min_pred) != 0)[0], len(first_5min_pred)-1])
class_id, class_count = np.unique(first_5min_pred[idx_end], return_counts=True)
print(f'{behavior_names[class_of_interest]} has {class_count[class_of_interest]} bouts within {start/10}s-->{stop/10}s')

face groom has 10 bouts within 0.0s-->300.0s


In [23]:
fileoutname = str.join('', (infile.rpartition('DLC')[0], '_posthoc-predictions.npy'))
np.save(fileoutname, predictions)

In [53]:
predictions #  100ms

array([5., 5., 5., ..., 5., 5., 5.])

In [54]:
realtime_vector = np.load('../../../raw_data/realtime_vector.npy')

In [55]:
realtime_vector_s = realtime_vector*10

In [56]:
realtime_vector_int = np.int64(realtime_vector_s)

In [57]:
realtime_vector_int

array([  144,   187,   393,   411,   420,   430,   434,   440,   473,
         587,   620,   747,   750,   841,  1056,  1120,  1496,  1672,
        1692,  1696,  1819,  1824,  1954,  1984,  2109,  2155,  2161,
        2179,  2209,  2338,  2592,  2596,  2686,  2866,  2924,  3038,
        3131,  3213,  3249,  3316,  3321,  3325,  3328,  3342,  4028,
        4176,  4270,  4323,  4478,  4485,  4534,  4550,  4597,  4708,
        4755,  4794,  4845,  5002,  5170,  5281,  5299,  5757,  5931,
        6048,  6090,  6125,  7028,  7092,  7107,  7387,  7391,  7800,
        7952,  7954,  8108,  8246,  8429,  8593,  8597,  8793,  8999,
        9022,  9314,  9394,  9495,  9510,  9639,  9723,  9734, 10009,
       10046, 10062, 10325, 10327, 10358, 10399, 10516, 10524, 10667,
       10712, 11085, 11239, 11302, 11309, 11351, 11433, 11465, 11469,
       11572, 11752, 11754, 12359, 12439, 12729, 12821, 12855, 13310,
       13353, 13433, 13551, 13590, 13679, 13821, 13834, 13848, 14659,
       14722, 14729,

In [58]:
[predictions[i] for i in realtime_vector_int+3]

[5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 2.0,
 1.0,
 2.0,
 1.0,
 5.0,
 2.0,
 5.0,
 5.0,
 1.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 1.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 0.0,
 5.0,
 5.0,
 2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 2.0,
 1.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 3.0,
 4.0,
 4.0,
 4.0,
 3.0,
 3.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 0.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0

In [59]:
len(np.where(np.array([predictions[i] for i in realtime_vector_int])==2)[0])/len([predictions[i] for i in realtime_vector_int])

0.06338028169014084

In [60]:
np.where(np.array([predictions[i] for i in realtime_vector_int])==3)[0]

array([ 85, 100, 106, 109, 110, 143, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 180, 194, 195, 196, 202, 203, 227], dtype=int64)

In [61]:
len([predictions[i] for i in realtime_vector_int])

284

In [62]:
print(first_5min_pred[-87:])

[5. 1. 1. 5. 5. 5. 5. 5. 5. 1. 1. 1. 1. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 2. 5. 5. 5. 5. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]


In [63]:
first_5min_pred = predictions[:3000] # first 5 minutes (300*10 at 10fps)
print(first_5min_pred.shape, np.diff(first_5min_pred).shape)
idx_end = np.hstack([np.where(np.diff(first_5min_pred) != 0)[0], len(first_5min_pred)-1])

# np.

(3000,) (2999,)


In [64]:
predictions

array([5., 5., 5., ..., 5., 5., 5.])

In [65]:
since_40min_pred = predictions[15000:18000] # first 5 minutes (300*10 at 10fps)
print(since_40min_pred.shape, np.diff(since_40min_pred).shape)
idx_end = np.hstack([np.where(np.diff(since_40min_pred) != 0)[0], len(since_40min_pred)-1])

# np.

(3000,) (2999,)


In [66]:
9000/(10*60)

15.0

In [67]:
len(idx_end)

292

In [68]:
first_5min_pred[idx_end[-2:]]

array([5., 5.])

In [69]:
35/50

0.7

In [70]:
30/38

0.7894736842105263

In [71]:
71/190

0.3736842105263158

In [72]:
np.unique(first_5min_pred[idx_end], return_counts=True)

(array([0., 1., 2., 5.]), array([  2,   6,  38, 246], dtype=int64))

In [73]:
np.unique(since_40min_pred[idx_end], return_counts=True)

(array([0., 1., 2., 3., 4., 5.]),
 array([ 16,  10,  25,  80,  21, 140], dtype=int64))

In [74]:
print(predictions[:160])

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 3. 5. 3. 5. 3. 3. 3. 3. 3. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]


In [75]:
print(predictions[170:210])

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 0. 0.
 5. 5. 0. 5. 5. 5. 0. 0. 0. 0. 5. 5. 5. 5. 5. 5.]


In [76]:
print(predictions[395:400])

[5. 5. 2. 2. 2.]


In [77]:
print(predictions[405:415])

[2. 5. 2. 2. 5. 5. 2. 2. 2. 5.]


In [78]:
print(predictions[415:430])

[2. 2. 5. 5. 2. 5. 2. 2. 2. 2. 5. 5. 5. 5. 5.]


In [79]:
transition_frame = np.where(np.diff(predictions)!=0)[0]
predictions[transition_frame[0]]


5.0

## put the below code in another notebook 
#### loop through all .npy files once analyzed

In [ ]:
all_predictions = []
for _ in range(6):
    # join duplicates of the same array
    predictions = np.load(fileoutname)
    all_predictions.append(predictions)
# all_predictions
all_predictions_arr = np.hstack(all_predictions)

In [ ]:
class_of_interest=17
jitter=0.5
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hlines(class_of_interest-jitter, 0, len(all_predictions_arr), 'r', alpha=0.7)
ax.hlines(class_of_interest+jitter, 0, len(all_predictions_arr), 'r', alpha=0.7)
ax.vlines(0-jitter, class_of_interest-jitter, class_of_interest+jitter, 'r', alpha=0.7)
ax.vlines(len(all_predictions_arr)+jitter, class_of_interest-jitter, class_of_interest+jitter, 'r', alpha=0.7)
ax.scatter(np.arange(len(all_predictions_arr)), all_predictions_arr, s=0.5, color='k', alpha=0.3)
ax.set_xticks(np.arange(0, len(all_predictions_arr), 300))
ax.set_xticklabels(np.arange(0, len(all_predictions_arr)/10, 30))
ax.set_ylabel('behavior ID')
ax.set_xlabel('time (s)')
fig.savefig('./test.png', dpi=600)

In [ ]:
all_predictions_arr.shape

In [ ]:
bin_start = np.arange(0, np.max(len(all_predictions_arr)), 600)
bin_end = np.arange(600, np.max(len(all_predictions_arr))+600, 600)
print(bin_start, bin_end)
binned_predictions = []
for b in range(len(bin_start)):
    binned_predictions.append(all_predictions_arr[bin_start[b]:bin_end[b]])


In [ ]:
class_interest_binned = [len(np.where(binned_predictions[b] == class_of_interest)[0]) for b in range(len(binned_predictions))]

plt.bar(np.arange(len(class_interest_binned)), class_interest_binned)

In [ ]:
np.arange(0, len(predictions), 300), np.arange(0, len(predictions)/10, 30)